In [1]:
# [OpenCV] 이미지에서 관심영역 설정하기 (좌표 입력, 마우스 드래그)

# https://hyongdoc.tistory.com/355

In [2]:
# OpenCV를 이용해 이미지에서 관심영역을 설정하는 방법

# 순서

# 1. 이미지의 좌표를 직접 설정해서 관심영역 표시하기
# 2. 설정된 관심영역으로 이미지 수정하기 (복제)
# 3. 마우스 드래그로 관심영역 설정하기


In [3]:
# 이미지에서 좌표값 입력을 통해 관심영역 표시하기

# 먼저 이미지를 불러온 뒤, 특정 좌표에 대한 값을 코드로 미리 설정함으로써
# 관심영역으로 지정하는 방법이다.
# 직관적이지만, 실제 사용을 위해서는 제한사항이 많아보이는 방법이다.

In [13]:
import cv2

img = cv2.imread('./data/test.jpg')

x = 300
y = 100
w = 150
h = 150
roi = img[y:y+h, x:x+w]

# print(roi.shape)
cv2.rectangle(roi, (0,0), (h-1,w-1), (0,255,0), 3)

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 먼저 원하는 지역의 좌측 상단에 해당하는 좌표값과, 지정하고자 하는
# 영역의 폭과 높이를 각각 지정한다.
# 그 후 roi에 이미지의 해당영역을 설정하고,
# 테두리 선을 둘러줍니다.
# rectangle을 통해 간단하게 직사각형 형태의 선을 표시할 수 있다
# 적용 시 이미지에는 아래와 같이 테두리 선이 나타나게 됨

In [14]:
# 관심영역을 통해 이미지 수정하기

# 위 예시에서 태양을 관심영역으로 지정했다.
# 이번에는 태양을 바로 옆에 복제시켜보도록 하겠다.
# 이미지는 어쨌거나 rgb에 해당하는 다차원 행렬데이터이기 때문에,
# 태양 옆의 영역에, 태양과 동일한 수치들로 이루어진 행렬만
# 복붙해주면 되는 원리이다.
# 코드는 다음과 같다.

In [21]:
import cv2

img = cv2.imread('./data/test.jpg')

x = 300
y = 0
w = 150
h = 150

roi = img[y:y+h, x:x+w]
img2 = roi.copy()

img[y:y+h, x+w:x+w+w] = roi
cv2.rectangle(img, (x,y), (x+w+w, y+h), (0,255,0))

cv2.imshow('img', img)
cv2.imshow('roi', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
# 마우스 드래그를 통해 관심영역 설정하기

# 이번에는 이미지를 띄운 뒤, 사용자가 마우스 드래그를 통해 관심영역을 설정하는 방법이다.
# 아무래도 위 방법들보다는 직관적이고 사용성이 좋은 듯 하다.
# 원리는 콜백함수를 이용한다.
# 콜백함수는 마우스 인풋을 이용하는 방식으로, 아래 코드를 보시면 직관적인 이해가 가능함
# 사용자가 마우스를 누르거나, 드래그하거나, 떼거나 하면 콜백함수가 실행되고
# 그에 따라 조건 별로 코드를 작성함

In [26]:
# https://inhovation97.tistory.com/57

import cv2

isDragging = False  # 마우스 드래그 상태 저장
x0, y0, w, h = -1,-1,-1,-1  # 영역 선택 좌표 저장
blue, red = (255,0,0), (0,0,255)  # 색상 값

def onMouse(event, x, y, flags, paam):  # 마우스 이벤트 핸들 함수
    global isDragging, x0, y0, img  # 전역 변수 참조
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼쪽 마우스 버튼 다운, 드래그
        # 시작
        isDragging = True  
        x0 = x
        y0 = y
    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 움직임
        if isDragging:  # 드래그 진행 중
            img_draw = img.copy()  # 사각형 그림 표현을 위한 이미지 복제(매번 같은 이미지에 그려지면 이미지가 더러워짐)
            cv2.rectangle(img_draw, (x0, y0), (x, y), blue, 2)  # 드래그 진행 영역 표시
            cv2.imshow('img', img_draw)  # 사각형으로 표시된 그림 화면 출력

    elif event == cv2.EVENT_LBUTTONUP:  # 왼쪽 마우스 버튼 업
        if isDragging:  # 드래그 중지
            isDragging = False
            w = x- x0   # 드래그 영역 폭 계산
            h = y- y0   # 드래그 영역 높이 계산
            if w > 0 and h > 0:  # 폭과 높이가 양수이면 드래그 방향이 옳음
                img_draw = img.copy()  # 선택 영역에 사각형 그림을 표시할 이미지 복제
                cv2.rectangle(img_draw, (x0, y0),(x, y), red, 2)  # 선택 영역에 빨간색 사각형 표시
                cv2.imshow('img', img_draw)   # 빨간색 사각형이 그려진 이미지 출력
                roi = img[y0:y0+h, x0:x0+w]  # 원본 이미지에서 선택 ROI로 지정
                cv2.imshow('cropped', roi)  # ROI 지정 영역을 새 창으로 표시
                cv2.moveWindow('cropped', 0,0)  # 새 창을 화면 좌측 상단으로 이동
                cv2.imwrite('./cropped.png', roi)  # ROI 영역만 파일로 저장

            else:
                # 드래그 방향이 잘못된 경우 사각형 그림이 없는 원본 이미지 출력
                cv2.imshow('img', img)
                print('drag should start from left-top side')

img = cv2.imread('./data/test.jpg')
cv2.imshow('img', img)
cv2.setMouseCallback('img', onMouse)  # 마우스 이벤트 등록
cv2.waitKey()
cv2.desrtoyAllWindows()

drag should start from left-top side
drag should start from left-top side
drag should start from left-top side
drag should start from left-top side
drag should start from left-top side


AttributeError: module 'cv2' has no attribute 'desrtoyAllWindows'

: 

In [2]:
# onMouse 첫 if문 부터 보자.

# 마우스 왼쪽 버튼이 눌리면, isDragging 상태를 바꿔주고 클릭한 좌표를 기억함
# 이 좌표는 관심영역의 좌상단이어야함

# 그 다음 마우스가 눌린 상태로 움직이면, 이미지를 복제해줌
# 계속 같은 이미지에 그리면 box가 중첩돼 더러워짐
# 복제한 이미지에 ROI를 그려줌

# 마우스 왼쪽 버튼을 떼면, isDragging 상태를 다시 바꿔주고 클릭했을 때의
# 좌표와 뗐을 때의 좌표를 계산하여 폭과 높이를 구해줌.
# 박스 친 ROI를 따로 저장해주고 이미지 창도 띄우게 됨

# 지정 경로에 가면 드래그한 ROI 이미지까지 저장 돼있는 것을 알 수 있다.
# 좌상단에서 우하단으로 드래그하지 않으면, 제대로하라는 메시지가 뜬다.

In [9]:
# 1. cv2라이브러리로 간단히 하기

import cv2, numpy as np

img = cv2.imread('./data/test.jpg')

x,y,w,h = cv2.selectROI('img', img, False)
if w and h:
    roi = img[y:y+h, x:x+w]
    cv2.imshow('cropped', roi)   # ROI 지정 영역을 새창으로 표시
    cv2.moveWindow('cropped', 0, 0)   # 새창을 화면 측 상단으로 이동
    cv2.imwrite('./data/cropped2.jpg', roi)  # ROI 영역만 파일로 저장

cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# cv2.selectROI([win_name,], img[,showCrossHair=True, fromCenter=False]) - ROI 지정

# 입력 :
# win_name - ROI 선택을 진행할 창의 이름, str
# img - ROI 선택을 진행할 이미지, numpy ndarray
# showCrossHair - 선택 영역 중심에 십자 모양 표시 여부
# fromCenter - 마우스 시작 지점을 여역의 중심으로 지정
# ret - 선택한 영역 좌표와 크기(x,y,w,h), 선택을 취소한 경우 모두 0


In [ ]:
# 해당 코드는 ROI를 마우스로 드래그한 뒤 엔터나 스페이스 바를 누르면,
# 지정한 경로로 크롭된 이미지가 저장되고,
# c키를 누르면 종료됨

# 

In [11]:
# 키보드, 마우스 이벤트 처리

# https://inhovation97.tistory.com/53

In [12]:
# 2장 기본 입출력
# 해당 실습은 cv2 라이브러리를 이용하여 진행함

# 1. 창 관리
# 2. 키보드 이벤트 처리
# 3. 마우스 이벤트 처리
# 4. 마우스 이벤트 처리 2
# 5. 트랙바 활용

In [3]:
# 1. 창 관리

import cv2

file_path = './data/test.jpg'
img = cv2.imread(file_path)  # 이미지 기본 값으로 읽기
img_gray = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 그레이스케일로 읽기

cv2.namedWindow('origin', cv2.WINDOW_AUTOSIZE)  # origin이라는 이름으로 창 생성
cv2.namedWindow('gray', cv2.WINDOW_NORMAL)  # gray라는 이름으로 창 생성

cv2.imshow('origin', img)  # origin 창에 이미지 표시
cv2.imshow('gray', img_gray)  # gray 창에 이미지 표시

cv2.moveWindow('origin', 0, 0)  # 창 위치 변경
cv2.moveWindow('origin', 100, 100)  # 창 위치 변경

cv2.waitKey(0)  # 아무키나 누르면
cv2.resizeWindow('origin', 200,200)  # 창 크기 변경 (변경 안됨)
cv2.resizeWindow('gray', 100,100)  # 창 크기 변경(변경 됨)

# cv2.waitKey(0)  # 아무키나 누르면
# cv2.destroyAllWindows('gray')  # gray 창 닫기

cv2.waitKey(0)  # 아무키나 누르면
cv2.destroyAllWindows()  # 모든 창 닫기

In [ ]:
# 위 코드를 실행하면, origin 이라는 이미지 창, gray라는 흑백 이미지 창이 2개
# 뜨는데 이때 아무키나 누르면 origin 창은 크기가 변경되지 않지만 gray 창은 크기가 변경됨

# 다시 한 번 아무키나 누르면 gray창이 먼저 닫히고 키를 한 번 더 누르면 origin 창 마저 닫히게 됨

In [ ]:
# cv2.namedWindow(title, [,option]) - 이름을 갖는 창 열기

# 입력:
# title - 제목
# option - 창 옵션, cv2.WINDOW_로 시작
# cv2.WINDOW_NORMAL - 임의의 크기, 사용자 창 크기 조정 가능
# cv2.WINDOW_AUTOSIZE - 이미지와 같은 크기, 창 크기 조정 불가능

# cv2.moveWindow(title, x, y) - 창 위치 이동
# 입력:
# title - 위치를 변경할 창의 이름
# x,y - 이동할 창의 위치

# cv2.resizeWindow(title, w, h) - 창 크기 변경
# 입력:
# title - 위치를 변경할 창의 이름
# w, h - 변경하고 싶은 창의 width, height

# cv2.destroyWindow(title) - 창 닫기
# 입력: title - 닫고 싶은 창의 이름

# cv2.destroyAllWindow() - 열린 창 모두 닫기

In [7]:
# 2. 키보드 이벤트 처리

import cv2

file_path = './data/test.jpg'
img = cv2.imread(file_path)
title = 'IMG'
x, y = 100, 100

while True:
    cv2.imshow(title, img)
    cv2.moveWindow(title, x, y)
    key = cv2.waitKey(0) & 0xFF  # 키보드 입력을 무한 대기, 8비트 마스크 처리
    print(key, chr(key))  # 키보드 입력 값, 문자 값 출력
    if key == ord('a'):
        x -= 10   # a라면 좌로 이동

    elif key == ord('s'):
        y += 10   # s라면 아래로 이동

    elif key == ord('w'):
        y -= 10   # w라면 위로 이동

    elif key == ord('d'):
        x += 10   # d라면 우로 이동

    elif key == ord('q') or key == 27:
        break   # q이거나 esc이면 종료
        cv2.destroyAllWindows()
    cv2.moveWindow(title, x, y)   # 새로운 좌표로 창 이동

0  
0  
0  
0  
0  
0  
0  
0  
100 d
100 d
100 d
119 w
119 w
119 w
97 a
97 a
100 d
115 s
100 d
115 s
100 d
115 s
100 d
100 d
100 d
100 d
97 a
100 d
97 a
100 d
97 a
100 d
97 a
100 d
97 a
100 d
97 a
255 ÿ


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1515: error: (-27:Null pointer) NULL window: 'IMG' in function 'cvMoveWindow'


In [ ]:
# 이번 코드는 이미지 창을 띄운 뒤 지정 키를 누르면 이미지가 코드 좌표상으로 10만큼씩 상하좌우로 움직이는 이벤트를 처리하는 것이다.
# key = cv2.waitKey(0) & 0xFF 에서
# cv2.waitKey(0)는 상시 대기하면서 어떤 키가 입력될 때를 기다리는 코드이다.
# 이때 이 함수가 출력하는 값이 ASCII 코드이다.
# 8비트인 아스키 코드를 반환하면, 그 반환 key 값을 ord() 함수를 씌어 문자열을 비교하게 되는 것이다.
# 아래 if문들을 보면 전부 ord() 함수를 씌운 문자열과 key 값을 비교함

# 하지만 이렇게 비교할 때 가끔씩 64비트 환경에서 cv2.waitKey(0) 함수가
# 32비트 정수를 반환하는 경우도 있다고 한다.
# 따라서 무조건 8비트를 만들어 주기 위해 뒤에 & 0xFF 연산자를 붙여 key 값을 반환함
# 그럼 이제 ord()문자열과 비교에 에러가 생기지 않는 것이다.
# 한글 모드에서 키를 입력하면 오류가 발생할 수 있으니 한글은 지양하는 것이 좋다고 함

In [7]:
# 3. 마우스 이벤트 처리

import cv2

title = 'mouse event'
img = cv2.imread('./data/test.jpg')
cv2.imshow(title, img)

def onMouse(event, x, y, flags, param):
    print(event, x, y)
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x, y), 30, (0,0,0), -1)
        cv2.imshow(title, img)

cv2.setMouseCallback(title, onMouse)  # 마우스 콜백 함수를 GUI 윈도우에 등록

while True:
    if cv2.waitKey(0) & 0xFF == 27:
        break
cv2.destroyAllWindows()

0 21 118
0 84 116
0 155 116
0 224 120
0 292 132
0 359 147
0 385 159
0 402 168
0 405 169
0 406 169
0 407 169
0 408 169
0 413 169
0 420 169
0 432 169
0 438 171
0 444 174
0 450 178
0 453 182
0 456 185
0 458 187
0 461 190
0 463 194
0 466 196
0 469 199
0 470 201
0 474 203
0 475 204
0 477 204
0 477 204
1 477 204
4 477 204
0 477 204
0 479 206
0 482 209
0 485 214
0 489 218
0 497 227
0 502 233
1 502 233
0 506 235
0 509 238
0 512 240
0 514 242
0 516 242
0 516 243
0 517 243
0 518 244
0 519 244
0 521 245
4 521 245
0 525 246
0 525 246
0 526 247
0 531 249
0 536 249
0 540 250
0 549 251
0 557 252
0 563 253
0 575 255
0 584 257
1 584 257
0 592 260
0 598 261
0 601 262
0 604 262
0 604 262
0 605 262
0 606 263
0 609 263
0 612 264
0 615 264
4 615 264
0 615 264
0 624 266
0 633 267
0 641 268
0 656 271
0 668 271
0 687 273
0 701 275
0 715 277
1 715 277
0 731 279
0 743 281
0 748 282
0 758 283
0 764 284
0 767 284
0 770 284
0 770 284
0 771 284
0 772 284
0 774 284
0 775 284
4 775 284
0 778 284
0 778 284
0 778 284
0 

: 

In [ ]:
# 마우스에서 입력을 받으려면 이벤트를 처리할 함수(onMouse 함수)를 미리
# 선언해놓고 cv2.setMouseCallback() 함수에 그 함수를 전달함
# 해당 코드는 마우스 왼쪽 버튼을 클릭하면 까만 원이 그려짐

In [ ]:
# cv2.setMouseCallback(win_name, onMouse [,param]) - onMouse 함수를 등록

# 입력:
# win_name - 이벤트를 등록할 윈도 이름
# onMouse - 이벤트 처리를 위해 미리 선언해 놓은 콜백 함수
# param - 필요에 따라 onMouse 함수에 전달할 인자

In [ ]:
# MouseCallback(event, x, y, flags, param) - 콜백 함수 선언부
# 입력:
# event - 마우스 이벤트 종류, cv2.EVENT_로 시작하는 상수 (12가지)
# cv2.EVENT_MOUSEMOVE - 마우스 움직임
# cv2.EVENT_LBUTTONDOWN - 왼쪽 버튼 누름
# cv2.EVENT_RBUTTONDOWN - 오른쪽 버튼 누름
# cv2.EVENT_MBOTTONDOWN - 가운데 버튼 누름
# cv2.EVENT_LBOTTONUP - 왼쪽 버튼 뗌
# cv2.EVENT_RBOTTONUP - 오른쪽 버튼 뗌
# cv2.EVENT_MBOTTONUP - 가운데 버튼 뗌
# cv2.EVENT_LBOTTONBLUCK - 왼쪽 버튼 더블 클릭
# cv2.EVENT_RBOTTONBLUCK - 오른쪽 버튼 더블 클릭
# cv2.EVENT_MBPTTPNBLUCK - 가운데 버튼 더블 클릭
# cv2.EVENT_MOUSEWHEEL - 휠 스크롤
# cv2.EVENT_MOUSEHWHEEL - 가로 휠 스크롤
# x, y - 마우스 좌표
# 
# flags - 마우스 동작과 함께 일어난 상태, cv2.EVENT_FLAG_로 시작하는 상수(6가지)
# cv2.EVENT_FLAGS_LBOTTON(1) - 왼쪽 버튼 누름
# cv2.EVENT_FLAGS_RBOTTON(2) - 오른쪽 버튼 누름
# cv2.EVENT_FLAGS_MBOTTON(4) - 가운데 버튼 누름
# cv2.EVENT_FLAGS_LCTRLKEY(8) - 컨트롤 버튼 누름
# cv2.EVENT_FLAGS_SHIFTKEY(16) - 쉬프트 버튼 누름
# cv2.EVENT_FLAGS_ALTKEY(32) - 알트 버튼 누름

# param - cv2.setMouseCallback() 함수에서 전달한 인자

# 함수 내부에서 사용하지 않더라도 onMouse 함수는
# 5개의 인자를 모두 기재해야하며, 그렇지 않으면 오류가 발생함

In [2]:
# 4. 마우스 이벤트 처리 2

import cv2

title = 'mouse event'
img = cv2.imread('./data/test.jpg')
cv2.imshow(title, img)

colors = {'black':(0,0,0),
'red':(0,0,255),
'blue':(255,0,0),
'green':(0,255,0)}  # 색상 미리 정의

def onMouse(event, x, y, flags, param):
    print(event, x, y, flags)
    color = colors['black']
    if event == cv2.EVENT_LBUTTONDOWN:  # 왼족 버튼을 누른 경우
        # 컨트롤 & 쉬프트 키를 같이 누른 경우
        if flags & cv2.EVENT_FLAG_CTRLKEY and flags & cv2.EVENT_FLAG_SHIFTKEY:
            color = colors['green']

        # 쉬프트 키를 누른 경우
        elif flags & cv2.EVENT_FLAG_SHIFTKEY:
            color = colors['blue']

        # 컨트롤 키를 누른 경우
        elif flags & cv2.EVENT_FLAG_CTRLKEY:
            color = colors['red']

        # 지름 30크기의 검은색 원을 해당 좌표에 그림
        cv2.circle(img, (x,y), 30, color, -1)
        cv2.imshow(title, img)

cv2.setMouseCallback(title, onMouse)

while True:
    if cv2.waitKey(0) & 0xFF == 27:
        break

cv2.destroyAllWindows()

0 162 611 0
0 164 595 0
0 167 578 0
0 171 565 0
0 173 560 0
0 175 559 0
0 176 557 0
0 178 557 0
0 180 555 0
0 182 554 0
0 184 554 0
0 186 554 0
0 190 554 0
0 192 554 0
0 196 553 0
0 203 552 0
0 213 550 0
0 222 548 0
0 236 544 0
0 248 538 0
0 260 531 0
0 275 521 0
0 293 509 0
0 320 487 0
0 345 468 0
0 356 459 0
0 371 447 0
0 385 435 0
0 395 426 0
0 407 415 0
0 416 405 0
0 425 397 0
0 428 392 0
0 433 384 0
0 435 380 0
0 436 377 0
0 436 374 0
0 436 372 0
0 436 370 0
0 436 368 0
0 436 367 8
0 436 366 8
0 437 363 8
0 437 363 8
0 438 363 8
1 438 363 9
0 438 362 9
4 438 362 8
0 438 362 8
0 438 363 8
7 438 363 9
0 438 363 9
0 439 363 9
0 439 363 9
0 440 363 9
0 441 363 9
0 443 362 9
4 443 362 8
0 447 361 8
0 450 360 8
0 458 359 8
0 467 358 8
0 480 357 8
0 497 354 8
0 537 348 8
0 569 343 8
0 621 337 8
0 652 333 8
0 698 327 8
0 727 322 8
0 755 321 8
0 769 319 8
0 778 319 8
0 781 319 8
1 781 319 9
0 782 319 9
4 782 319 8
0 782 319 8
0 783 319 8
0 784 318 8
0 785 318 8
0 785 317 8
0 786 316 8
0 78

KeyboardInterrupt: 

In [3]:
# flags 값은 함께 누른 경우와 같이 여러 가지 상태를 하나의 값으로 한꺼번에 나타낼 수 있어야 함.
# 그래서 실제 갖는 값은 2진수 비트 자릿수에 맞는 값을 가짐
# (위에서도 써놨지만, cv2.EVENT_FLAGS_SHIFTKEY(16)-쉬프트 버튼 누름 -> 쉬프트 버튼 입력은 16임)
# 만약 flag 값이 홀수인 경우 (25=1+8+16) 어떤 플래그 상수인지 명확히 알 수 없어
# 위 코드처럼 if문을 짜야함
# 위 코드는 마우스 오른쪽 버튼을 클릭하면 검은색, 컨트롤 키를 누른 채로 클릭하면 빨간색,
# 시프트 키는 파란색, 시프트 + 컨트롤 키는 초록색 원을 그림

In [10]:
# 5. 트랙바 활용

import cv2 
import numpy as np

win_name = 'Trackbar'

img = cv2.imread('./data/test.jpg')
cv2.imshow(win_name, img)

# 트랙바 이벤트 처리 함수 선언
def onChange(x):
    print(x)
    # R G B 각 트랙바 위치 값
    r = cv2.getTrackbarPos('R', win_name)
    g = cv2.getTrackbarPos('G', win_name)
    b = cv2.getTrackbarPos('B', win_name)
    print(r, g, b)
    img[:] = [b,g,r]
    cv2.imshow(win_name, img)

# 트랙바 생성
cv2.createTrackbar('R', win_name, 0, 255, onChange)
cv2.createTrackbar('G', win_name, 0, 255, onChange)
cv2.createTrackbar('B', win_name, 0, 255, onChange)

while True:
    if cv2.waitKey(1) & 0xFF == 27:
        break
cv2.destroyAllWindows()

1
0 0 1
0
0 0 0


In [6]:
# 트랙바는 슬라이드 모양의 인터페이스를 마우스로 움직여서 값을 입력받는 GUI 요소이다.

# cv2.createTrackbar(trackbar_name, win_name, value, count, onChange) - 트랙바 생성
# 입력:
# trackbar_name - 트랙바 이름
# win_name - 트랙바를 표시할 창 이름
# value - 트랙바 초기 값, 0~count 사이의 값
# count - 트랙바 눈금의 개수, 트랙바 이벤트 핸들러 함수
# onChange - TrackbarCallback, 트랙바 이벤트핸들러 함수

# TrackbarCallback(value) - 트랙바 이벤트 콜백 함수
# 입력:
# trackbar_name - 트랙바 이름

# CV2.getTrackbarPos(trackbar_name, win_name) - 트랙바의 현재 위치를 출력
# 입력: 
# trackbar_name - 찾고자 하는 트랙바 이름
# win_name - 트랙바가 있는 창의 이름
# pos - 트랙바 위치 값

In [ ]:
# 파이썬 동영상 파일 & 웹캠 읽기, 저장하기

# 해당 실습은 cv2 라이브러리를 이용하여 진행함

# 1. 동영상 읽기
# 2. 웹캠 켜기
# 3. 웹캠 크기 조절하기
# 4. 웹캠 이미지 저장하기
# 5. 웹캠 동영상 저장하기

In [ ]:
# 1. 동영상 읽기

# 동영상은 이미지 프레임 하나하나를 연속적으로 읽어오는 원리이다.
# 그 속도가 영상의 속도를 좌우하는데, 보통의 동영상은 40fps(frams per second)인 경우가 가장 많다고 함
# fps에 맞는 지연속도를 구하여 프레임이 들어가는 속도를 조절하는데,
# 그 계산은 아래와 같다.

# 지연시간 = 1000/fps
# 1초를 밀리초 단위로 환산해서 제공해야하기에 1000으로 계산함
# 지연시간이 길다는 건 fps값이 작아 프레임을 넣는 시간이 길어지는 것이다.